In [ ]:
%pwd

In [ ]:
%%capture
!pip install transformers[sentencepiece]
import transformers

Processing the data.

In [ ]:
PATH_TO_FOLER = "../Name2Demographics/"

In [ ]:
!cd $PATH_TO_FOLDER

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


import sys  
sys.path.insert(1, '../Name2Demographics/Models/CBSEData/CBSEPreprocessing/')
from parse_cbse import *

In [5]:
cbse_df = CBSEData()

In [6]:
cbse_df = cbse_df[:100]

In [7]:
%%capture
!pip install datasets

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)


from transformers import AutoConfig
config = AutoConfig.from_pretrained("vinai/bertweet-base")

In [9]:
# train = pd.read_csv("Models/Data/CBSEData/cbse_train.csv")
# test = pd.read_csv("Models/Data/CBSEData/cbse_test.csv")

# X_train = list(train['Name'])
# X_val = list(map(str,test['Name']))
# y_train = list(map(int, train['Gender']))
# y_val = list(map(int, test['Gender']))

In [10]:
from sklearn.model_selection import train_test_split

X = list(cbse_df["Name"])
y = list(cbse_df["Gender"].values.astype('int'))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

X_train_tokenized = tokenizer(X_train, padding=True, truncation=True)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True)

In [11]:
import torch

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [12]:
from transformers import TrainingArguments
# from transformers import Trainer

In [13]:
from transformers import Trainer

In [14]:
import os

wandb.init(mode='disabled')

NameError: name 'wandb' is not defined

In [15]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import EarlyStoppingCallback


# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f_recall = recall_score(y_true=labels, y_pred=pred, pos_label=0)
    f_precision = precision_score(y_true=labels, y_pred=pred, pos_label=0)
    f_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=0)
    m_recall = recall_score(y_true=labels, y_pred=pred, pos_label=1)
    m_precision = precision_score(y_true=labels, y_pred=pred, pos_label=1)
    m_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=1)
    return {"accuracy": accuracy, "female_precision": f_precision, "female_recall":f_recall, "female_f1": f_f1,
        "male_precision": m_precision, "male_recall": m_recall, "male_f1": m_f1 }

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=50,
    per_device_eval_batch_size=50,
    num_train_epochs=1,
    seed=42,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()

/home/souvic/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vahini10 (use `wandb login --relogin` to force relogin)


/home/souvic/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


TrainOutput(global_step=1, training_loss=0.7042415738105774, metrics={'train_runtime': 9.0071, 'train_samples_per_second': 7.772, 'train_steps_per_second': 0.111, 'total_flos': 323749931400.0, 'train_loss': 0.7042415738105774, 'epoch': 1.0})

In [16]:
metrics=trainer.evaluate()
print(metrics)

/home/souvic/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


{'eval_loss': 0.5205672979354858, 'eval_accuracy': 1.0, 'eval_female_precision': 1.0, 'eval_female_recall': 1.0, 'eval_female_f1': 1.0, 'eval_male_precision': 0.0, 'eval_male_recall': 0.0, 'eval_male_f1': 0.0, 'eval_runtime': 0.2188, 'eval_samples_per_second': 137.093, 'eval_steps_per_second': 4.57, 'epoch': 1.0}


/home/souvic/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/souvic/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/souvic/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [17]:
metrics

{'eval_loss': 0.5205672979354858,
 'eval_accuracy': 1.0,
 'eval_female_precision': 1.0,
 'eval_female_recall': 1.0,
 'eval_female_f1': 1.0,
 'eval_male_precision': 0.0,
 'eval_male_recall': 0.0,
 'eval_male_f1': 0.0,
 'eval_runtime': 0.2188,
 'eval_samples_per_second': 137.093,
 'eval_steps_per_second': 4.57,
 'epoch': 1.0}

In [ ]:
# model.save_pretrained("Models/CBSEData/BERT/SavedModel/BERT")

In [ ]:
%pwd

%%capture
!pip install transformers[sentencepiece]
import transformers

Processing the data.

PATH_TO_FOLER = "../Name2Demographics/"

!cd $PATH_TO_FOLDER

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


import sys  
sys.path.insert(1, '../Name2Demographics/Models/CBSEData/CBSEPreprocessing/')
from parse_cbse import *

cbse_df = CBSEData()

cbse_df = cbse_df[:100]

%%capture
!pip install datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")
model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base", num_labels=2)


from transformers import AutoConfig
config = AutoConfig.from_pretrained("vinai/bertweet-base")

# train = pd.read_csv("Models/Data/CBSEData/cbse_train.csv")
# test = pd.read_csv("Models/Data/CBSEData/cbse_test.csv")

# X_train = list(train['Name'])
# X_val = list(map(str,test['Name']))
# y_train = list(map(int, train['Gender']))
# y_val = list(map(int, test['Gender']))

from sklearn.model_selection import train_test_split

X = list(cbse_df["Name"])
y = list(cbse_df["Gender"].values.astype('int'))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

X_train_tokenized = tokenizer(X_train, padding=True, truncation=True)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True)

import torch

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

from transformers import TrainingArguments
# from transformers import Trainer

from transformers import Trainer

import os

wandb.init(mode='disabled')

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import EarlyStoppingCallback


# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f_recall = recall_score(y_true=labels, y_pred=pred, pos_label=0)
    f_precision = precision_score(y_true=labels, y_pred=pred, pos_label=0)
    f_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=0)
    m_recall = recall_score(y_true=labels, y_pred=pred, pos_label=1)
    m_precision = precision_score(y_true=labels, y_pred=pred, pos_label=1)
    m_f1 = f1_score(y_true=labels, y_pred=pred,pos_label=1)
    return {"accuracy": accuracy, "female_precision": f_precision, "female_recall":f_recall, "female_f1": f_f1,
        "male_precision": m_precision, "male_recall": m_recall, "male_f1": m_f1 }

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=50,
    per_device_eval_batch_size=50,
    num_train_epochs=1,
    seed=42,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()

metrics=trainer.evaluate()
print(metrics)

metrics

# model.save_pretrained("Models/CBSEData/BERT/SavedModel/BERT")



